In [3]:
from openai import OpenAI
import os
from OpenAI_API_KEY import key
import pandas as pd

In [11]:

API_URL = 'https://api.openai.com/v1/engines/gpt-3.5-turbo/completions'

client = OpenAI(api_key = key)

def gpt_brief(county):
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a analytical graphing assistant."},
      {"role": "user", "content": f"Give me a brief analysis of the employment environment of {county}."}
    ]
  )
  return(completion.choices[0].message.content)

html_chart_template_example = """<body>
    <div class="chart-container">
        <canvas id="EmploymentBarChart"></canvas>
    </div>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script>
        const data = {
            labels: [
                "Total Nonfarm",
                "Service-Providing",
                "Total Wage and Salary",
                "Total Private",
                "Total State and Local Government",
                "Total Private",
                "Government",
                "Local Government",
                "Private Service Providing",
                "Retail Trade",
                "Professional and Business Services",
                "Trade, Transportation, and Utilities",
                "Goods Producing",
                "Leisure and Hospitality",
                "Private Education and Health Services",
                "Mining, Logging and Construction",
                "Non-Durable Goods",
                "Manufacturing",
                "Financial Activities",
                "Construction",
                "Information",
                "Mining and Logging",
                "State Government",
                "Total Farm",
                "Wholesale Trade",
                "Transportation, Warehousing, and Utili...",
                "Durable Goods",
                "Other Services",
                "Federal Government"
            ],
            datasets: [{
                label: 'Industry Current Employment',
                data: [12430, 10780, 12690, 7620, 4740, 7620, 4810, 2750, 5960, 1400, 560, 1680, 1660, 1270, 1630, 860, 600, 800, 250, 570, 160, 290, 1990, 260, 60, 220, 200, 430, 70],
                backgroundColor: 'rgba(54, 162, 235, 0.2)',
                borderColor: 'rgba(54, 162, 235, 1)',
                borderWidth: 1
            }]
        };

        const config = {
            type: 'bar',
            data: data,
            options: {
                indexAxis: 'y',
                elements: {
                    bar: {
                        borderWidth: 2,
                    }
                },
                responsive: true,
                plugins: {
                    legend: {
                        position: 'top',
                    }
                }
            },
        };

        window.onload = function() {
            const ctx = document.getElementById('EmploymentBarChart').getContext('2d');
            new Chart(ctx, config);
        };
    </script>
</body>
"""
def gpt_bar_chart(data):
  completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a analytical graphing assistant."},
    {"role": "user", "content": f"Generate a sorted (from high to low) html bar chart with the given dataframe using leaflet. Only give me the body section of the html without a title or header.{data} use the following html template as example.{html_chart_template_example}"}
  ]
  )
  return(completion.choices[0].message.content)

In [14]:
employment_breif = []
location_df = pd.read_csv("../data/Location_df_v2.csv")
for index, row in location_df.iterrows():
    employment_breif.append(gpt_brief(row['Area Name']))
employment_breif_df = pd.DataFrame(employment_breif)

In [20]:
employment_breif_df = employment_breif_df.astype(str)
employment_breif_df.to_csv("../data/employment_summary_df.csv")


In [9]:
county_employment_df = pd.read_csv("../data/updated_employment_data.csv")
current_employment_df = county_employment_df.loc[(county_employment_df["Year"] == 2024)&(county_employment_df["Month"] == "April")]
current_location_list = current_employment_df["Area Name"].unique()
html_list = []
len(current_location_list)

74

In [12]:
for county in current_location_list:
    per_county_employment_df = current_employment_df[current_employment_df['Area Name'] == county][['Industry Title','Industry Current Employment']]
    html_list.append(gpt_bar_chart(per_county_employment_df))


In [28]:
gpt_bar_df = pd.DataFrame({"Area Name": current_location_list, "Chart": html_list})
gpt_bar_df["Chart"] = gpt_bar_df["Chart"].astype(str)

In [35]:
gpt_bar_df.to_csv("../data/html_chart.csv", index = False)

In [37]:
gpt_chart_df = pd.read_csv("../data/html_chart.csv")
gpt_chart_df

,Area Name,Chart
0,Alpine County,"\n<body>\n <div class=""chart-container"">\n ..."
1,Amador County,"\n<body>\n <div class=""chart-container"">\n ..."
2,Butte County,"<body>\n <div class=""chart-container"">\n ..."
3,Calaveras County,"\n<body>\n <div class=""chart-container"">\n ..."
4,Colusa County,"\n<body>\n <div class=""chart-container"">\n ..."
...,...,...
69,Stockton-Lodi MSA,"\n<body>\n <div class=""chart-container"">\n ..."
70,Vallejo-Fairfield MSA,"\n<body>\n <div class=""chart-container"">\n ..."
71,Visalia-Porterville MSA,"\n<body>\n <div class=""chart-container"">\n ..."
72,Yuba City MSA,"\n<body>\n <div class=""chart-container"">\n ..."
